# INF200 Lecture No 9

### 8 November 2021

## Today's topics

1. Object-oriented programming
    1. Review
    1. Defining new data types
1. Staying in control
    1. Assertions

# Object-oriented programming

----------

## Review

See slides for Lecture 8.

-------------------

## Defining new data types

- OO Idea 1: Combine data and behavior into *new data types*
- Problem: How to make our classes behave more like built-in data types
    - nice printing
    - comparison between instances (e.g., sorting `Member`s)
    - mathematical operations (e.g., computing with vectors)
- Solution: Operator overloading
- See, e.g., Langtangen ch 7.3-7.5 (4th edition)
- **Overloading**: Giving an operation a (new) meaning.

### Overloading in Python

- All classes inherit from `object` methods for
    - initialization (constructor)
    - string representation (printing)
    - comparison (by `id`)
    - etc
- Operations are implemented by `__xxxxxx__()` methods
- We can *overload* these functions to define behavior for our classes
- First example: constructor `__init__()`

#### Defining the string representation of objects

In [1]:
class Member:
    def __init__(self, name, number):
        self.name, self.number = name, number

    def display(self):
        print("Member: {0.name} (#{0.number})".format(self))
        
joe = Member('Joe', 123)
jane = Member('Jane', 456)

print(joe, jane)

<__main__.Member object at 0x7fb104fcd1c0> <__main__.Member object at 0x7fb104fcd220>


- Default string representation from `object`
- Not useful
- Add string representation methods `__str__()` and `__repr__()`

In [4]:
class Member:
    def __init__(self, name, number):
        self.name, self.number = name, number
        
    def __str__(self):
        return "Member: {0.name} (#{0.number})".format(self)

    def __repr__(self):
        return "Member('{0.name}', {0.number})".format(self)

    def display(self):
        print("Member: {0.name} (#{0.number})".format(self))
        
joe = Member('Joe', 123)
jane = Member('Jane', 456)

print(joe)
print(jane)
print([joe, jane])

Member: Joe (#123)
Member: Jane (#456)
[Member('Joe', 123), Member('Jane', 456)]


- The two string representation methods:
    - **`__str__()`**
        - called by `print` and `str` it it exists
        - should return "user friendly" display of instance
    - **`__repr__()`**
        - called in all other cases
        - also called by `print` and `str` if
            - `__str__()` is not defined
            - the instance is part of a list, tuple or dictionary
        - should return a string that can be used to recreate the object
    - Both methods must return a string
    - If you want to implement only one of the two, implement `__repr__()`
- We can re-define the `display()` method in terms of `__str__()`
    - Note that `print(self)` inside a method is equivalent to `print(self.__str__())`

In [5]:
class Member:
    def __init__(self, name, number):
        self.name, self.number = name, number
        
    def __str__(self):
        return "Member: {0.name} (#{0.number})".format(self)

    def __repr__(self):
        return "Member('{0.name}', {0.number})".format(self)

    def display(self):
        print(self)

- In subclasses, we now only need to override `__str__()` and `__repr__()`, but not `display()`

In [6]:
class Officer(Member):
    def __init__(self, name, number, rank):
        Member.__init__(self, name, number)
        self.rank = rank

    def __str__(self):
        return "{0.rank}: {0.name} (#{0.number})".format(self)

    def __repr__(self):
        return "Officer('{0.name}', {0.number}, '{0.rank}')".format(self)

jack = Officer('Jack', 789, 'President')

In [7]:
members = [joe, jane, jack]
print("Members as list:", members)
for member in members:
    member.display()

Members as list: [Member('Joe', 123), Member('Jane', 456), Officer('Jack', 789, 'President')]
Member: Joe (#123)
Member: Jane (#456)
President: Jack (#789)


### Defining mathematical operations

- `+`, `-`, `*`, `/` and further mathematical operators can be defined for classes
- See http://docs.python.org/library/operator.html for a complete list
- No default definitions are inherited from `object`: only what you provide is available
- Think carefully about what definitions may make sense, e.g.,
    - string addition: concatenation
    - string times integer n: concatenate string with itself n times
    - subtraction and division not definable for string
- Methods: `__add__`, `__sub__`, `__mul__`, `__truediv__`
- `a + b` is equivalent to `a.__add__(b)`
- Below
    - `lhs`: left-hand side
    - `rhs`: right-hand side

In [11]:
import math

In [15]:
class Vector:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'

    def __add__(self, rhs):
        return Vector(self.x + rhs.x, self.y + rhs.y)

    def __sub__(self, rhs):
        return Vector(self.x - rhs.x, self.y - rhs.y)

    def __mul__(self, rhs):
        return Vector(self.x * rhs, self.y * rhs)

    def __rmul__(self, lhs):
        return self * lhs

    def __truediv__(self, rhs):
        return self * (1. / rhs)

    def norm(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

We create a few vectors and work with them. Note that `print` now falls back on the `__repr__()` method for printing the vectors, because `__str__()` is not implemented.

In [17]:
v = Vector(1, 2)
w = Vector(30, 40)

print("v       = ", v)
print("w       = ", w)
print("v + w   = ", v + w)
print("v * 5   = ", v * 5)
print("2 * v   = ", 2 * v)
print("v / 10  = ", v / 10)
print("norm(v) = ", v.norm())

v       =  Vector(1, 2)
w       =  Vector(30, 40)
v + w   =  Vector(31, 42)
v * 5   =  Vector(5, 10)
2 * v   =  Vector(2, 4)
v / 10  =  Vector(0.1, 0.2)
norm(v) =  2.23606797749979


- `__rmul__()` vs `__mul__()`:
    - `v * 5` is `v.__mul__(5)`: no problem, run `Vector.__mul__(v, 5)`
    - `2 * v` would be `2.__mul__(v)`, i.e., `int.__mul__(2, v)`
    - `int` knows nothing about vectors: error!
    - `__rmul__()`: called with swapped arguments if `__mul__()` fails
    - `2 * v` becomes `v.__rmul__(2)`, running `Vector.__rmul__(v, 2)`
    - `__rmul__()` usually implemented in terms of `__mul__()` or `*`
- `r`-versions also for other math methods

### Overriding comparisons

- `<`, `<=`, `>`, `>=`, `==`, `!=` can be overriding by defining `__lt__`, `__le__`, `__gt__`, `__ge__`, `__eq__`, `__ne__`
- `x < y` is equivalent to `x.__lt__(y)`
- Shall return `True` or `False`
- This set of six comparisons is known as "rich comparisons"

#### Default comparisons

- By default, a new class inherits comparisons from the fundamental base class `object`
- `__eq__` and `__ne__` test for *object identity*
    - same `o1 == o2` means the same as `o1 is o2`
    - ususally sensible, except for mathematical types
- All other comparisons return `NotImplemented` and will result in an error

In [18]:
o1 = object()
o2 = object()

o1 == o1, o1 == o2

(True, False)

In [19]:
o1 < o2

TypeError: '<' not supported between instances of 'object' and 'object'

#### Class-specific comparisons

- Override only comparisons that can be defined meaningfully!
- Equality can be defined for most types
- Only define "less than" and similar if there is one universal way of ordering instances of a class
    - Numbers are well-ordered in a mathematical sense: define `__lt__()` etc
    - Vectors can only be compared for equality
    - If instances can be ordered in different ways in different situations (by name, member number, age, ...) define the ordering rule as `key` to the sorting function
- If you define "less than", implement all other comparisons as well
    - Define them in terms of `<` and `==` to ensure consistency
    
##### Example: Vector class

- Only equality and inequality
- Try first vector class from above *without* comparisons

In [20]:
v1 = Vector(1, 2)
v2 = Vector(1, 2)
v1 == v2

False

- Result is `False` because `Vector` inherited `__eq__` from `object` and tests for `v1 is v2`
- Now create class with overridden comparisons

In [22]:
class NewVector:

    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __eq__(self, rhs):
        return self.x == rhs.x and self.y == rhs.y
    
    def __ne__(self, rhs):
        return not self == rhs

    def __repr__(self):
        return 'Vector({0.x}, {0.y})'.format(self)

    def __add__(self, rhs):
        return Vector(self.x + rhs.x, self.y + rhs.y)

    def __sub__(self, rhs):
        return Vector(self.x - rhs.x, self.y - rhs.y)

    def __mul__(self, rhs):
        return Vector(self.x * rhs, self.y * rhs)

    def __rmul__(self, lhs):
        return self * lhs

    def __truediv__(self, rhs):
        return self * (1. / rhs)

    def norm(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

In [23]:
nv1 = NewVector(1, 2)
nv2 = NewVector(1, 2)
nv3 = NewVector(1.0, 2.0)
nv4 = NewVector(5, 8)

nv1 == nv2, nv1 == nv3, nv1 == nv4

(True, True, False)

- We now compare for equality in the mathematical sense
- We do not allow relative comparisons

In [24]:
nv1 < nv4

TypeError: '<' not supported between instances of 'NewVector' and 'NewVector'

#### Example: a fraction class supporting all comparisons

- Note that we only need to implement `__eq__()` and `__lt__()` explicitly
- All other comparisons can be constructed from those two.

In [31]:
class Fraction:
    def __init__(self, a, b):
        assert b > 0, "Denominator b > 0 required."
        self.a, self.b = a, b
    
    def __eq__(self, rhs):
        return self.a * rhs.b == rhs.a * self.b
    
    def __ne__(self, rhs):
        return not self == rhs
    
    def __lt__(self, rhs):
        return self.a * rhs.b < rhs.a * self.b  # expand fractions to same denominator and compare numerators

    def __le__(self, rhs):
        return self < rhs or self == rhs
    
    def __gt__(self, rhs):
        return rhs < self

    def __ge__(self, rhs):
        return rhs <= self

In [32]:
Fraction(1, 2) == Fraction(2, 4)

True

In [16]:
f1 = Fraction(3, 4)
f2 = Fraction(2, 3)

In [17]:
f1 == f2, f1 < f2, f1 >= f2

(False, False, True)

------------

# Staying in control

## Background

- Computers can solve complex tasks fast
- Humans tend to trust in results provided by computers
- In some situations, lives depend on computers working correctly
- Requires reliable software
- Difficult to achieve: we can demonstrate the presence of bugs, proving their absence is (essentially) impossible
- Field of software engineering: *Verification* and *Validation*
- We look only at essential elements

### Elements of reliable software

- Software shall not return incorrect results
- Software shall fail in controlled ways 
- Software shall handle unforseen conditions
- Software shall be tested solidly
- **Software should fail rather than return incorrect results.**

### Techniques towards reliability

- **Assertions**
    - check that requirements are fulfilled
    - stop execution if requirement not fulfilled
    - key use cases
        - very simple "emergency stops" if we don't want to spend time on proper error handling
        - catching things that "cannot happen", but where we want to be on the safe side (in large projects, you never know ...)
- **Exceptions**
    - mechanism for signaling that something unexpected happended
    - available in most modern programming languages
    - exceptions are *raised* or *thrown* when a problem is detected
    - exceptions can be *caught* and *handled*, e.g., by issuing a useful error message
    - in some languages, e.g., Python, exceptions are also used as part of normal programming
- **Testing**
    - systematic testing of code can help us to find errors
    - a proper set of tests also helps us to avoid introducing new errors as software evolves
    - *unit tests* are tests of small parts of code, typically functions
    - *integration tests* test that the parts of a larger project work together
    - *regression tests* are added when a bug is discovered
        - the test reproduces the bug
        - when the bug is fixed, the test passes
        - we keep the test, in case we should re-introduce the bug by a later change (regress)

----------

## Assertions

- *pass* if a boolean expression is True
- *fail* if a boolean expression is False

In [33]:
assert True

In [34]:
assert False

AssertionError: 

We can use them to catch certain conditions

In [35]:
def inverse(x):
    """Returns 1 / x."""
    
    assert x != 0
    
    return 1. / x

In [36]:
inverse(10)

0.1

In [37]:
inverse(0.)

AssertionError: 

We can provide some more information to the user by adding a string after the boolean expression:

In [38]:
def inverse(x):
    """Returns 1 / x."""
    
    assert x != 0, "Inverse of 0 is not defined."
    
    return 1. / x

In [39]:
inverse(0)

AssertionError: Inverse of 0 is not defined.

We can also use this to check for conditions that are mathematically defined, but make no sense.

In [40]:
import math

def area(r):
    """Returns area of circle with radius r."""
    
    assert r >= 0, 'Circle radius must be positive.'
    
    return math.pi * r**2

In [41]:
area(1)

3.141592653589793

In [42]:
area(-1)

AssertionError: Circle radius must be positive.